##### Quantum Data Science 2021/2022
# Lecture 8 - Quantum Feature maps, Kernels and Support Vector Machines

<!-- no toc -->
### Contents 

1. [Introduction](#intro)
2. [Support Vector Machines](#svm)
3. [Caveats](#caveats)
4. [Non-linear separation via feature maps](#non-linear)
5. [Support Vector Machines with Kernels](#kernels)
6. [Data encoding as a Quantum Feature map](#qfeaturemap)
7. [Quantum Kernel estimation](#qke)
8. [References](#references)

## 1 - Introduction <a id="intro"></a>

Suppose we have a label dataset dataset with $M$ points of 2 feature only , i.e $x_i \in \mathbb{R}^2$
$$ \mathcal{X} = \{(x_0^0, x_1^0), (x_0^1, x_1^1), \dots, (x_0^{M-1}, x_1^{M-1}) \}$$

with two classes: 
$$ \mathcal{Y} \in \{-1,1\}$$ 

Consider a 2D plot of the data with multiple decision boundaries:

<p align="center">
 <img width="800" height="400" src="images/decision_boundaries.jpeg">
</p>


<span style="color: red;">Question: </span> What hypothesis class should we choose? 

(3) looks very expressive, therefore we should choose it, right?
No, notice that every hypothesis correctly classify the entire dataset. Therefore, we should choose the minimum complexity hypothesis that works ! ---> **Occam's Razor**. This is again because of the problem of overfitting and generalization! 

So, a Linear classifier should be chosen! 

$$ \mathcal{H} = \{x \mapsto sign(w^T x + b) : x \in \mathbb{R}^2, w \in \mathbb{R}^2, b \in \mathbb{R} \}$$ 

The sign operator returns the sign of the operation inside the brackets. For simplicity we're considering $\mathbb{R}^2$, but this could be very well generalized to larger dimensions -> $\mathbb{R}^N$

Remember that in this case, the general hyperplane equation is:

$$ w.x + b = 0$$ 

<p align="center">
 <img width="500" height="500" src="images/r3.jpeg">
</p>


<span style="color: red;">Question: </span> But, there are infinite hyperplanes that correctly separate the two classes ! How do we choose the best ?


## 2 - Support Vector Machines <a id="svm"></a>

We should choose the model that minimizes the margin $\gamma$ i.e. the model with smallest distance between the hyperplane and a single point in the dataset.

<p align="center">
 <img width="600" height="500" src="images/support_vectors.jpg">
</p>

##### Calculating the margin:

Consider the following picture:

<p align="center">
 <img width="600" height="500" src="images/gamma.png">
</p>

Let $\frac{w}{||w||}$ be the unit length vector. We know that point B can be calculated as: 

$$A- \gamma \frac{w}{||w||}$$
for a point in the positive side of the hyperplane. Thus, putting back in the hyperplane and solving for $\gamma$ , with an arbitrary point $x_i$ leads to: 

$$\gamma_i = \frac{w^T x_i + b}{||w||}$$

In general, for any point, the margin can be calculated inserting the corresponding label:

$$\gamma_i = y_i(\frac{w^T x_i + b}{||w||})$$

Notice that $\gamma_i$ is always $>0$ once the prediction is correct, i.e fell on the correct side of the hyperplane . 

We can always restrict ourselves to margin being equal to one, simplifying the problem! 

<p align="center">
 <img width="800" height="400" src="images/margin=1.png">
</p>

Leading to the well known **primal formulation** for the optimization problem in support vector machines.

$$ max_{w,b} \frac{1}{||w||} = min_{w,b} \frac{1}{2} ||w||^2 $$

subject to constraints :

$$ y_i(w^T x_i + b) \geq 1 $$

This can be solved with off the shelf software! E.g Sklearn 

## 3 - Caveats <a id="caveats"></a>

* Differentiable function and $\nabla_w \frac{1}{2} ||w||^2 = w$ and $\nabla_w^2 \frac{1}{2} ||w||^2 = 1$
* Eigenvalues are $>0$ thus the function is **stricly convex**. This is awesome because it tell us that we have a unique solution - Any local minimum is the global minimum. This is not true is general neural networks algorithms. Here there are well established theoretical guarantees. For instance the the number of datapoints necessary to generalize and guarantee a minimum prediction error - Sample complexity. See [1]
  
<p align="center">
 <img width="800" height="400" src="images/convex.jpeg">
</p>

* However, recall that $w \in \mathbb{R}^N$. Thus when we have a million features , the optimization becomes prohibitive. Thus , the **dual formulation** is often used

$$\mathcal{L}(w,\alpha,b) = \frac{1}{2} ||w||^2 - \sum_{i=1}^{M} \alpha_i (y_i(w^T x_i +b) - 1)$$

considering: 

$$\nabla_{w} L=w-\sum_{i=1}^{M} \alpha_{i} y_{i} x_{i}=0 \quad \Rightarrow \quad w=\sum_{i=1}^{M} \alpha_{i} y_{i} x_{i}$$

$$\nabla_{b} v=-\sum_{i=1}^{M} \alpha_{i} y_{i}=0 \quad \Rightarrow \quad \sum_{i=1}^{M} \alpha_{i} y_{i}=0$$

$$\forall_i, \alpha_{i}\left[y_{i}\left(w^{\top}, x_{i}+b\right)-1\right]=0 \Rightarrow \alpha_{i} \approx 0 \vee y_{i}\left(w_{i} x_{i}+b\right)=1$$

Implying that now we just need to optimize as many $\alpha$'s as the number of training points in our dataset. Moreover, $\alpha = 1$ only for support vectors which are much fewer than the total number of points. 

Replacing $w$ and $b$, we get the most used expression for support vector machines: 

$$
\begin{array}{c}
\max _{\alpha} \mathcal{L}(\alpha) \\ 
\\
\max _{\alpha} \sum \alpha_{i}-\frac{1}{2} \sum_{i, j=1}^{M} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_i^T  x_j\right) \\
\\
\text { s.t } \alpha_{i} \geqslant 0 \wedge \sum_{i=1}^{M} \alpha_{i} y_{i}=0
\end{array}
$$

* Learning algorithms based on SVM's are efficiently learnable !!! In the language of **PAC Learning**
  
  --- 
  
  **PAC LEARNING:** 
  
  *Probably approximately correct* - The number of samples that we need to have an $\epsilon$-approximation of the empirical error. 

  $$ \mathbb{P} \left[ R \leq \epsilon\right] \geq 1 - \delta$$

  An algorithm is PAC Learnable if the number of samples grows only polylogarithmically with error, confidence interval, size of hypothesis ... 

  ---

Where $x_i^T x_i$ is the **inner product** between datapoints $i$ and $j$. 

<span style="color: red;">Question: </span> What if the data is not linearly separable? 

Enters Kernels ! 

## 4 - Non linearly separable data <a id="nonlinear"></a>

### Feature maps: 
The hope is that we can apply a non-linear transformation to the data, mapping our points to a larger dimension space $\mathcal{H}$ -> Feature space. 

$$ \phi: \mathcal{X} \mapsto \mathcal{H} $$

<p align="center">
 <img width="900" height="400" src="images/concentric circles.png">
</p>

<span style="color: red;">Question: </span> But the feature vectors will not become too big? Well, there are even infinite dimensional feature spaces. For instance, via an exponential feature map! 

We don't need to work with feature vectors provided we can estimate inner products! Provided we can compare vectors in large feature spaces! 

Enters kernels and the kernel trick !! 


### Kernels : 

The Kernel $K$ is a similarity measure between two points $x_i$ and $x_j$ in feature space: 

$$ K: \Omega \times \Omega \mapsto \mathbb{R} $$
$$ K: (x,y) \longmapsto k(x,y)=\langle\phi(x) , \phi(y)\rangle $$

The Kernel trick allow us to compare points in high dimensional feature spaces by using the original input vectors! 

**Mercer's condition**: $k(x,y) \geq 0$  and the kernel matrix $\mathcal{K}$:

$$ \mathcal{K} = \left[ k(x,y) \right]_{xy}$$ 

is positive semidefinite. Convexity !

Take a look at well-known Kernels:


<p align="center">
 <img width="/00" height="300" src="images/kernels.png">
</p>



## 5 - Support Vector Machines with Kernels <a id="kernels"></a>

We can use the dual formulation but now replacing the inner product with the kernel:

$$
\begin{array}{c}
\max _{\alpha} \mathcal{L}(\alpha) \\ 
\\
\max _{\alpha} \sum \alpha_{i}-\frac{1}{2} \sum_{i, j=1}^{M} \alpha_{i} \alpha_{j} y_{i} y_{j} \langle\phi(x) , \phi(y)\rangle \\
\\
\text { s.t } \alpha_{i} \geqslant 0 \wedge \sum_{i=1}^{M} \alpha_{i} y_{i}=0
\end{array}
$$


### <span style="color: red;">Question: </span>
 But wait a minute ? How do we choose these kernels ? Do we need to make the data to higher dimension feature spaces, or lower dimension? 

--- 
Answer: Lot's of engineering needed based on feature knowledge! 

---> This is the reason for Neural Networks being used nowadays , because they find non-linear decision boundaries on their own, without needing a human to design a kernel

---

## 6 - Quantum Feature maps and kernels <a id="qfeaturemap"></a>

### Goal: Represent the kernel function with a quantum computer! Why?
Because quantum computers already work in high dimensional feature spaces. 

We can simply use the data encoding as a feature map !

Then we just need to find a way of calculating inner products between point in feature space, which we already know how to do ! 

### Quantum Feature map:

$$ \phi: x \mapsto |\phi(x) \rangle $$

### Quantum kernel: 

For two points $x$ and $y$ , the kernel can be obtained from the well-known fidelity: 

$$k(x,y) = |\langle \phi(x) | \phi(y) \rangle|^2 $$

This is indeed a kernel. Notice that this is also positive semidefinite ( $k(x,y) \geq 0 $)


<p align="center">
 <img width="/00" height="300" src="images/kernel_types.png">
</p>

<span style="color: red;">Question: </span> How can we calculate the fidelity? 

Can we do better? 

<span style="color: red;">Exercise: </span> Derive a quantum circuit to estimate the fidelity. Compare with the swap test


<p align="center">
 <img width="600" height="300" src="images/inverse_test.png">
</p>


The point is: Why do we need a quantum computer to estimate the kernels described above? Yes, ESTIMATE! Why do we estimate? 

Well, the answer is ... we don't!! Classical computers can efficiently compute said kernels. Therefore, the goal here is to exploit quantum processes that are not efficiently simulatable by a classical computer and create powerful feature maps! 

One such example is the IQP encoding: 

<p align="center">
 <img width="800" height="250" src="images/IQP.png">
</p>

which is conjectured to be hard to simulate classically, as we increase the number of layers. 

### GOAL : Design feature maps that are intractable to simulate classically

## 7 - Quantum Kernel estimation <a id="qke"></a>


As we noticed above, if we measure the probability of basis state $|0\rangle$ , we measure the kernel ! 

$$ \mathbb{P}(|0\rangle) = |\langle \phi(x) | \phi(y) \rangle|^2 $$



Since we need many shots to estimate the probability, depending on the type of kernel, the number of shots may remove any quantum advantage. However, it seems that we can estimate only with additive sampling error, which is good! 

## 8 - References <a id="references"></a>

* [1] - Mohri, Mehryar et al. “Foundations of Machine Learning.” Adaptive computation and machine learning (2012).

Ciao for now :) 